# Preparation
Here we prepare directories and a File with links if the environment ist new

In [1]:
drivepath = './'

In [3]:
#Prepares the directories for the PDF and TXT files
import os
drivepath = './'

pdf_directory = drivepath+'Downloads_BOA/'
txt_directory = drivepath+'Downloads_TXT/'

# Create the directories if they do not exist
os.makedirs(pdf_directory, exist_ok=True)
os.makedirs(txt_directory, exist_ok=True)




We prepare our Transformers

In [4]:
#Install the library if environment is not already set up
!pip install sentence_transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Michael\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Next we load a pretrained transformer. We can experiment here with different models.

In [5]:
# 1. Load a pretrained Sentence Transformer model
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-large") #  We could use different models here
#other examples are: "all-MiniLM-L6-v2" "T-Systems-onsite/cross-en-de-roberta-sentence-transformer" "intfloat/multilingual-e5-large"
#model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')


C:\Users\Michael\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In new environments install pdfreader

In [6]:
!pip install pdfreader


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Michael\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


We extract the texts from the PDF documents. In the interest of transparency, we store these in TXT files per page. 
Additionally, an array with all texts and references to PDF/pages is created.

In [9]:
import os
import pdfreader
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

import time

# Directory containing the PDF files
pdf_directory = drivepath + 'Downloads_BOA/'
txt_directory = drivepath + 'Downloads_TXT/'

# Initialize a variable to store text from the current PDF
text = np.array([])

# Function to process a single PDF file
def process_pdf(filename):
    local_text = []
    if filename.endswith('.pdf'):
        # Open the PDF file
        with open(os.path.join(pdf_directory, filename), 'rb') as pdf_file:
            # Create a PDF reader object
            pdf_reader = pdfreader.SimplePDFViewer(pdf_file)
            doc = pdfreader.PDFDocument(pdf_file)
            
            # Iterate through all the pages and extract text
            all_pages = [p for p in doc.pages()]
            pdf_title = pdf_reader.metadata.get('Title', '')

            for i in range(len(all_pages)):   #iterate through all pages
                pdf_reader.navigate(i + 1)
                pdf_reader.render()
                page_text = ''
                with open(os.path.join(txt_directory, f"{filename}-{i}.txt"), "w") as f: #save text to txt file
                    for line in pdf_reader.canvas.strings:
                        try:
                            f.write(line)
                            page_text += line
                            if '.' in line and page_text[-4:] != 'Mio.': #if line ends with a dot, it is the end of a sentence and a new line is started
                                f.write('\n')
                                local_text.append({'text': str(page_text), 'filename': filename, 'page': i + 1, 'Titel': pdf_title})
                                page_text = ''
                        except Exception as e:
                            print(f"Error processing line in {filename} page {i}: {e}")
                    f.close()
    return local_text



# Define the number of CPUs to use
num_cpus = os.cpu_count()  # or set a specific number, e.g., num_cpus = 4

# Use ThreadPoolExecutor to parallelize the processing of PDF files
with ThreadPoolExecutor(max_workers=num_cpus) as executor:
    # Submit the PDF processing tasks
    futures = [executor.submit(process_pdf, filename) for filename in os.listdir(pdf_directory)]

    
    for future in as_completed(futures):
        text = np.append(text, future.result())
    


# text now contains the text of all PDFs in the directory
# more preprocessing can be done here, e.g. removing stopwords, stemming, etc.

Error processing line in 8.00 Reglement Markt- Zins- und Liquiditätsrisikomanagement 2020.pdf page 0: 'charmap' codec can't encode character '\uf0ea' in position 0: character maps to <undefined>
Error processing line in 8.00 Reglement Markt- Zins- und Liquiditätsrisikomanagement 2020.pdf page 0: 'charmap' codec can't encode character '\uf0ea' in position 0: character maps to <undefined>
Error processing line in 8.00 Reglement Markt- Zins- und Liquiditätsrisikomanagement 2020.pdf page 0: 'charmap' codec can't encode character '\uf0ea' in position 0: character maps to <undefined>
Error processing line in 7.00 Rahmenkonzept Risikomanagement.pdf page 2: 'charmap' codec can't encode character '\uf07d' in position 0: character maps to <undefined>
Error processing line in 7.00 Rahmenkonzept Risikomanagement.pdf page 2: 'charmap' codec can't encode character '\uf07d' in position 0: character maps to <undefined>
Error processing line in 7.00 Rahmenkonzept Risikomanagement.pdf page 2: 'charmap' 

reread Text-Files 

In [10]:
print(text)

[{'text': '          Münzverarbeitung GST     Rubrik   Allgemein  Gültig ab  01.', 'filename': 'Münzverarbeitung GST.pdf', 'page': 1, 'Titel': ''}
 {'text': '01.', 'filename': 'Münzverarbeitung GST.pdf', 'page': 1, 'Titel': ''}
 {'text': '2023 Bewilligungsstelle bei  der Bank  Risk Officer der Bank  Datum der Bewilligung bei der Bank  31.', 'filename': 'Münzverarbeitung GST.pdf', 'page': 1, 'Titel': ''}
 ...
 {'text': ' Eine Ausnahme bilden die Amortisationen von grundpfandgesicherten Krediten.', 'filename': '4.10 Kredithandbuch.pdf', 'page': 72, 'Titel': ''}
 {'text': ' Für Kreditgeschäfte, welche vor dem 01.', 'filename': '4.10 Kredithandbuch.pdf', 'page': 72, 'Titel': ''}
 {'text': '04.', 'filename': '4.10 Kredithandbuch.pdf', 'page': 72, 'Titel': ''}]


In [7]:
import numpy as np
import os
text = np.array([])
for filename in os.listdir('Weisungen_TXT/'):
    with open(os.path.join('Weisungen_TXT/',filename),"r") as file:
        for line in file:
            text = np.append(text,{'text':line,'filename':filename})
print(text)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 88: invalid start byte

Calculate the embeddings from the source files

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # For progress bar

# Function to encode a batch of sentences along with their indices
def encode_batch(batch):
    indices, sentences = zip(*batch)
    embeddings = model.encode(sentences)
    return list(zip(indices, embeddings))

# The sentences to encode
sentences = [(i, textlines['text']) for i, textlines in enumerate(text)]  # List of (index, sentence) tuples

# Batch processing
batch_size = 20  # Adjust based on your memory and model capacity
sentence_embeddings = []

# Use ThreadPoolExecutor to parallelize the batch processing
with ThreadPoolExecutor() as executor:
    futures = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        futures.append(executor.submit(encode_batch, batch))
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        batch_embeddings = future.result()
        sentence_embeddings.extend(batch_embeddings)

# Add embeddings to the text array
for index, embedding in sentence_embeddings:
    text[index]['embedding'] = embedding

# Now the text array has embeddings added to each entry

100%|██████████| 1311/1311 [07:28<00:00,  2.92it/s]


In [86]:
print(text)


[{'text': '         2.0', 'filename': '2.00 Anhang Homeoffice.pdf', 'page': 1, 'Titel': '', 'embedding': array([ 0.00184272,  0.00732435, -0.01063247, ..., -0.01172565,
        -0.03823478,  0.02175484], dtype=float32)}
 {'text': '0  Personalreglement der Bank - Anhang Homeoffice     Rubrik   Personal  Gültig ab  01.', 'filename': '2.00 Anhang Homeoffice.pdf', 'page': 1, 'Titel': '', 'embedding': array([ 0.00723311, -0.00773406, -0.02969398, ..., -0.0160033 ,
        -0.05287737, -0.02007233], dtype=float32)}
 {'text': '06.', 'filename': '2.00 Anhang Homeoffice.pdf', 'page': 1, 'Titel': '', 'embedding': array([ 0.01351172,  0.03566608,  0.00069096, ...,  0.01448118,
        -0.02628937,  0.00907879], dtype=float32)}
 ...
 {'text': '3.', 'filename': 'Leitfaden Informationssicherheit.pdf', 'page': 17, 'Titel': '', 'embedding': array([ 0.02574197,  0.0144252 , -0.01804409, ..., -0.01563678,
        -0.05338345, -0.00392191], dtype=float32)}
 {'text': '1 Nachweis über die Funktionsfähigkei

In [12]:
from sklearn.metrics.pairwise import cosine_similarity


# The sentences to encode
sentences = []
for textlines in text:
    sentences.append(textlines['text']) #we write the text from the PDFs into the sentences array

sentence_embeddings = model.encode(sentences) #calculate the embeddings of the sentences

Now we can check the result

In [29]:
print(text_ordered)

[{'text': ' Seite 1/4 Clientis AG Gurtengasse 6 Postfach  3001 Bern  Telefon 031 660 46 44   info@clientis.\n', 'filename': 'cli-mm-ab-240314-jahresergebnis-de.pdf-0-.txt'}
 {'text': 'ch clientis.\n', 'filename': 'cli-mm-ab-240314-jahresergebnis-de.pdf-0-.txt'}
 {'text': 'ch  Medienmitteilung    Bern, 14.\n', 'filename': 'cli-mm-ab-240314-jahresergebnis-de.pdf-0-.txt'}
 {'text': ' März 2024 Ein sehr erfolgreiches Jahr dank Kontinuität und regionaler Verankerung  Die 14 Regionalbanken der Clientis Gruppe haben im Geschäftsjahr 2023 das nachhaltige Wachstum im Kundengeschäft fortgesetzt und die starke Eigenkapitalbasis erneut verbessert (+4,7%).\n', 'filename': 'cli-mm-ab-240314-jahresergebnis-de.pdf-0-.txt'}
 {'text': ' Die Hypothekarforderungen stiegen um 4,1% auf CHF 11,2 Milliarden.\n', 'filename': 'cli-mm-ab-240314-jahresergebnis-de.pdf-0-.txt'}
 {'text': ' Der Gruppengewinn liegt 8,8% höher bei CHF 69,6 Mio. \n', 'filename': 'cli-mm-ab-240314-jahresergebnis-de.pdf-0-.txt'}
 {'text'

In [14]:
np.save('text.npy', text) #save the text array to a file

In [17]:
query = ['welche belehnungssätze sind bei einfamilienhaus massgebend?'] # This is the sentence we want to compare to the others
#exaples: 'Mitarbeitende Clientis Banken und Standorte' 'Bilanzsumme 2023' 'CEO der Clientis AG' 'Clientis Gruppe Mitglieder' 'Entwicklung Gewinn' 'Cost / Income Ratio (vor Abschreibungen)'
query_embeddings = model.encode(query)
sentence_embeddings = [(i, textlines['embedding']) for i, textlines in enumerate(text)]  # List of (index, sentence) tuples
sentence_embeddings = np.array([embedding for index, embedding in sentence_embeddings])

similarities = cosine_similarity(query_embeddings, sentence_embeddings)
top_similarities = similarities[0].argsort()[-3:][::-1] # Get the top 3 most similar sentences
for pos in top_similarities:
    print(pos)
    print(f"Sentence: {text[pos]['text']}")
    print(f"Sentence before: {text[pos-1]['text']}")
    print(f"Sentence after: {text[pos+1]['text']}")
    print(f"Source: {text[pos]['filename']} page {text[pos]['page']}") #
    print(f"Similarity: {similarities[0][pos]}")
    
    print()



2842
Sentence:  Belehnungssätze ................................
Sentence before:  1 7.
Sentence after: ................................
Source: 4.00 Kreditreglement.pdf page 2
Similarity: 0.8950374722480774

24884
Sentence:  Massgebend sind die Belehnungssätze, welche durch die vorliegende Weisung vorgegeben werden.
Sentence before:  Säule und der Säule 3a sind ihrer Zusammensetzung entsprechend (Kontoguthaben, Police, Wert-schriften) belehnbar und zu gewichten.
Sentence after:         4.
Source: 4.10 Kredithandbuch.pdf page 26
Similarity: 0.8840814232826233

25266
Sentence: 2 Kalkulatorische Sätze für die Tragbarkeitsberechnung  Grundstückstyp Kalkulatorischer Zinssatz  Kalkulatorische Nebenkosten in % der "Massgeblichen Belehnungsbasis" Einfamilienhaus (inkl.
Sentence before: 3.
Sentence after:  Zweifamilienhaus) 5% 0.
Source: 4.10 Kredithandbuch.pdf page 37
Similarity: 0.8815066814422607



In [15]:
print(query_embeddings)

[[ 0.0121971   0.00563831 -0.03830355 ... -0.01959545 -0.04714379
  -0.01946857]]


After all the processing, we should clean our desk :)

In [ ]:
import os

folder_path = drivepath + 'Weisungen_PDF/'
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    os.remove(file_path)
folder_path = drivepath + 'Weisungen_TXT/'
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    os.remove(file_path)

In [47]:
similarities.shape
similarities.transpose().shape


NameError: name 'scores' is not defined